<a href="https://colab.research.google.com/github/MaxVortman/IDEA-code-clones/blob/master/colab/code_clones_bag_of_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
%cd /content/gdrive/My\ Drive/code-clones

/content/gdrive/My Drive/code-clones


In [9]:
!ls

all_tests.txt  data	metrics.py   test.txt	v0.2.0.zip
csv	       log.txt	__pycache__  train.txt	xmlextractor.py


In [0]:
import xmlextractor
import metrics
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

In [0]:
with open('csv/duplicates.pkl', 'rb') as f:
  duplicates = pickle.load(f)

In [0]:
dupBoW = []
for duplicate in duplicates:
    text = '\n\n'.join(duplicate.fragments)
    dupBoW.append({'text': text, 'exp': duplicate.exp})

In [15]:
df = pd.DataFrame(dupBoW)
df.head()

,exp,text
0,0,/**\n * A hash map using primitive ints as key...
1,0,\tprivate static ByteBuffer readImage(final Bu...
2,0,\t\tSpriteRendererTF() {\n\t\t\tSystem.out.pri...
3,0,\t/**\n\t * Processes keyboard input\n\t */\n\...
4,0,"if (!VertLeq(o1, d1)) {\n G..."


In [16]:
df.describe()

,exp
count,19990.000000
mean,0.271636
std,0.444814
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [0]:
vect = CountVectorizer(ngram_range=(1,1))
bagOfWords = vect.fit_transform(df['text'])

In [0]:
from scipy import sparse

In [0]:
y = sparse.coo_matrix(df['exp'])

In [0]:
sparse.save_npz("csv/bagofwords_vectors_X.npz", bagOfWords)
sparse.save_npz("csv/bagofwords_vectors_y.npz", y)

In [0]:
sparse_bow_X = sparse.load_npz("csv/bagofwords_vectors_X.npz")
sparse_bow_y = sparse.load_npz("csv/bagofwords_vectors_y.npz").T

In [31]:
%cd / 

/


In [35]:
tsvd = TruncatedSVD(n_components=65, random_state=23, n_iter=10)
X = tsvd.fit_transform(sparse_bow_X)
sum(tsvd.explained_variance_ratio_)

0.9900126162151632

In [36]:
!git clone https://github.com/hyperopt/hyperopt-sklearn
%cd hyperopt-sklearn
!pip install -e .

fatal: destination path 'hyperopt-sklearn' already exists and is not an empty directory.
/hyperopt-sklearn
Obtaining file:///hyperopt-sklearn
  Found existing installation: hpsklearn 0.0.3
    Can't uninstall 'hpsklearn'. No files were found to uninstall.
  Running setup.py develop for hpsklearn


In [37]:
from hpsklearn import HyperoptEstimator, any_sparse_classifier, any_classifier
from hyperopt import tpe

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [0]:
def custom_loss(y_target, y_prediction):
  k_fn = 1
  k_fp = 10
  n = len(y_target)
  (fnr, fpr) = metrics.calc_fnr_fpr(y_target, y_prediction)
  return k_fn * fnr + k_fp * fpr

In [0]:
def resample(X, y):
  rus = RandomUnderSampler(random_state=23)
  return rus.fit_resample(X, y)

In [0]:
X_resampled, y_resampled = resample(X, sparse_bow_y.toarray())
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=23)

In [55]:
estim = HyperoptEstimator(classifier=any_classifier('clf'),
                          preprocessing=[],
                          algo=tpe.suggest, 
                          trial_timeout=300,
                          loss_fn=custom_loss,
                          seed=23)
estim.fit(X_train, y_train)

100%|██████████| 1/1 [00:11<00:00, 11.47s/it, best loss: 1.09085328422345]


In [56]:
y_pred = estim.predict(X_test)
metrics.print_fnr_fpr(y_test, y_pred)
metrics.print_metrics(y_test, y_pred)


FNR: 
0.16427255985267036

FPR: 
0.12670349907918968

accuracy: 
0.7090239410681399

precision: 
0.7314597970335676

recall: 
0.6775126536514823


In [57]:
estim.best_model()

{'ex_preprocs': (),
 'learner': GradientBoostingClassifier(criterion='friedman_mse', init=None,
                            learning_rate=0.005891891362147046, loss='deviance',
                            max_depth=None, max_features='log2',
                            max_leaf_nodes=None, min_impurity_decrease=0.0,
                            min_impurity_split=None, min_samples_leaf=8,
                            min_samples_split=2, min_weight_fraction_leaf=0.0,
                            n_estimators=16, n_iter_no_change=None,
                            presort='auto', random_state=0, subsample=1.0,
                            tol=0.0001, validation_fraction=0.1, verbose=0,
                            warm_start=False),
 'preprocs': ()}

In [67]:
thr = 0.5
y_prob = [y[1] for y in estim.best_model()['learner'].predict_proba(X_test)]
y_pred = [int(y >= thr) for y in y_prob]
_, fpr = metrics.calc_fnr_fpr(y_test, y_pred)
while fpr - 0.10 >= 0.005:
  thr += 0.0005
  y_pred = [int(y >= thr) for y in y_prob]
  _, fpr = metrics.calc_fnr_fpr(y_test, y_pred)

ValueError: ignored

In [68]:
thr

0.5

In [76]:
metrics.print_roc_auc(y_test, y_prob)
metrics.print_fnr_fpr(y_test, y_pred)
metrics.print_metrics(y_test, y_pred)


roc auc: 
0.8059990576259557

FNR: 
0.19631675874769797

FPR: 
0.10460405156537753

accuracy: 
0.6990791896869245

precision: 
0.7495590828924162

recall: 
0.6146059291395517


In [0]:
pipe = Pipeline([('tsvd', tsvd), ('clf', estim.best_model()['learner'])])

In [75]:
%cd /content/gdrive/My\ Drive/code-clones
with open('models/tsvd_bow.model', 'wb') as f:
  pickle.dump(pipe, f)

/content/gdrive/My Drive/code-clones
